<a href="https://colab.research.google.com/github/Rishabh5752/Text_Attack_Exp/blob/main/BERT_base_uncased_AGNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 115.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 13.1 MB/s e

In [2]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 52.7 MB/s eta 0:00:00


In [3]:
import textattack
import transformers

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [5]:
# Construct our four components for `Attack`

from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwap

In [6]:
goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)

constraints = [

    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.7)

]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:37<00:00, 13.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpxt11cun9.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [7]:
from textattack.search_methods import GreedySearch

In [8]:
transformation = textattack.transformations.word_swaps.word_swap_embedding.WordSwapEmbedding()
search_method = textattack.search_methods.GreedySearch()

In [9]:
#attack = textattack.attack_recipes.deepwordbug_gao_2018.DeepWordBugGao2018.build(model_wrapper)
attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

#datasets.load_dataset('emotion', split =['test'])
#dataset=datasets.load_dataset('sms_spam', split =['test'])
dataset = textattack.datasets.HuggingFaceDataset("ag_news", dataset_columns=(["text"],'label'))
#attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset ag_news, split train.


In [10]:
attack_args = textattack.AttackArgs(

    num_examples=100,
    disable_stdout=True

)

attacker = textattack.Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 89 / 10 / 1 / 100: 100%|██████████| 100/100 [47:39<00:00, 28.59s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 89     |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 99.0%  |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 89.9%  |
| Average perturbed word %:     | 10.01% |
| Average num. words per input: | 37.75  |
| Avg num queries:              | 871.88 |
+-------------------------------+--------+


In [11]:
from textattack import Attacker

attacker = Attacker(attack, dataset)
attack_results=attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



 10%|█         | 1/10 [00:06<00:59,  6.67s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:07<01:04,  7.14s/it]

[[Business (96%)]] --> [[World (55%)]]

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, [[Wall]] Street's dwindling\band of ultra-cynics, are seeing [[green]] again.

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, [[Mural]] Street's dwindling\band of ultra-cynics, are seeing [[environmentalist]] again.




...

--------------------------------------------- Result 8 ---------------------------------------------
[[Business (100%)]] --> [[World (77%)]]

[[Fed]] minutes show dissent over inflation (USATODAY.com) USATODAY.com - Retail [[sales]] [[bounced]] back a bit in [[July]], and new claims for [[jobless]] benefits fell [[last]] week, the government said [[Thursday]], [[indicating]] the economy is [[improving]] from a midsummer slump.

[[Fueled]] minutes show dissent over inflation (USATODAY.com) USATODAY.com - Retail [[salts]] [[banged]] back a bit in [[Avril]], and new claims for [[unemployed]] benefits fell [[another]] week, the government said [[Mardi]], [[depicting]] the economy is [[betterment]] from a midsummer slump.




[Succeeded / Failed / Skipped / Total] 9 / 0 / 0 / 9:  90%|█████████ | 9/10 [06:15<00:41, 41.71s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Business (100%)]] --> [[Sci/tech (64%)]]

Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in [[Sociology]], Danny Bazil Riley started to [[work]] as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a [[financial]] planner [[stopped]] by his [[desk]] to drop off brochures about [[insurance]] benefits available through his employer. But, at 32, "[[buying]] [[insurance]] was the furthest thing from my mind," says Riley.

Safety Net (Forbes.com) Forbes.com - After earning a PH.D. in [[Sociological]], Danny Bazil Riley started to [[collaborate]] as the general manager at a commercial real estate firm at an annual base salary of  #36;70,000. Soon after, a [[funding]] planner [[ceased]] by his [[desktop]] to drop off brochures about [[security]] benefits available through his employer. But, at 32, "[[procurement]] [[assuranc

[Succeeded / Failed / Skipped / Total] 10 / 0 / 0 / 10: 100%|██████████| 10/10 [06:22<00:00, 38.30s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Business (100%)]] --> [[World (100%)]]

Wall St. Bears Claw Back Into the Black  [[NEW]] YORK (Reuters) - Short-sellers, [[Wall]] Street's dwindling  band of ultra-cynics, are seeing green again.

Wall St. Bears Claw Back Into the Black  [[NOVEL]] YORK (Reuters) - Short-sellers, [[Mural]] Street's dwindling  band of ultra-cynics, are seeing green again.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 10     |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 14.6%  |
| Average num. words per input: | 38.5   |
| Avg num queries:              | 1149.2 |
+--------------------